# Chapter 23 - RFormula

In [1]:
%run common.ipynb

Name: Error parsing magics!
Message: Magics [run] do not exist!
StackTrace: 

In [2]:
%ShowTypes on

Types will be printed.


# Setup

In [3]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row

import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame

import org.apache.spark.ml.feature._

import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

In [4]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<!-- To left align the HTML components in Markdown -->

### Spark parition control based on core availability

In [5]:
val NUM_CORES = 4
val NUM_PARTITIONS = 3

lazy val spark: SparkSession = SparkSession.builder()
    .appName("mllib-rformula")
    .getOrCreate()

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/
import spark.implicits._

NUM_CORES: Int = 4
NUM_PARTITIONS: Int = 3
spark: org.apache.spark.sql.SparkSession = <lazy>


spark: org.apache.spark.sql.SparkSession = <lazy>


In [6]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,192.168.0.27)
(spark.eventLog.enabled,true)
(spark.driver.port,39237)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://192.168.0.27:39237/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-03b30901-c411-4496-81c7-4776d1f72153/repl-e77ab862-d6a4-4da6-be98-524cfa3614e2)
(spark.app.name,mllib-rformula)
(spark.driver.memory,3g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,12)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://192.168.0.27:4040)
(spark.org.apache

configMap: Unit = ()


## Constants

In [7]:
val PROTOCOL="file://"
val DATA_DIR="/home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data"
val RESULT_DIR="."

PROTOCOL: String = file://
DATA_DIR: String = /home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data
RESULT_DIR: String = .


RESULT_DIR: String = .


# Dataframe

In [8]:
var df = spark.read.json(PROTOCOL + DATA_DIR + "/simple-ml")

df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


df: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]


In [9]:
df.orderBy("value2").show(false)

+-----+----+------+------------------+
|color|lab |value1|value2            |
+-----+----+------+------------------+
|green|good|1     |14.386294994851129|
|green|bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|green|bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |good|35    |14.386294994851129|
|red  |bad |2     |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|green|good|1     |14.386294994851129|
|green|good|12    |14.386294994851129|
|blue |bad |8     |14.386294994851129|
|red  |good|35    |14.386294994851129|
|blue |bad |12    |14.386294994851129|
|red  |bad |16    |14.386294994851129|
|green|good|12    |14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



# Split train/test data

In [10]:
val Array(train, test) = df.randomSplit(Array(0.7, 0.3))

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]


test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [color: string, lab: string ... 2 more fields]


# Pipeline

## RFormula

In [11]:
val formula = new RFormula()
  .setFormula("lab ~ . + color:value1 + color:value2")

formula: org.apache.spark.ml.feature.RFormula = RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_5af6e3758e1f)


formula: org.apache.spark.ml.feature.RFormula = RFormula(lab ~ . + color:value1 + color:value2) (uid=rFormula_5af6e3758e1f)


## Logistic regression

In [12]:
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features")

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_9caa141f2e29


lr: org.apache.spark.ml.classification.LogisticRegression = logreg_9caa141f2e29


## Pipeline

In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
val pipeline = new Pipeline()
    .setStages(Array(
        formula,
        lr
    ))

val model = pipeline.fit(train)

## Model performance evaluation

In [ ]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val evaluator = new BinaryClassificationEvaluator()
  .setMetricName("areaUnderROC")
  .setRawPredictionCol("prediction")
  .setLabelCol("label")

evaluator.evaluate(model.transform(test))